Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install

In [21]:
!git clone https://github.com/davexhardware/CycleGAN-VAE-NST

fatal: destination path 'CycleGAN-VAE-NST' already exists and is not an empty directory.


In [2]:
import os
os.chdir('CycleGAN-VAE-NST/')

In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.8 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sh

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos]`

Or use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import zipfile
import glob

!unzip -d /trainB -o /content/drive/MyDrive/trainB_pt_16_128.zip
!unzip -d /trainA /content/drive/MyDrive/trainA_pt_16_128.zip

File_paths = glob.glob('/trainA')

Output streaming troncato alle ultime 5000 righe.
  inflating: /trainA/trainA_pt_16_128/063226.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063237.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063240.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063260.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063265.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063267.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063301.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063305.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063306.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063323.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063328.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063342.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063344.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063375.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063395.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063405.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063425.jpg.pt  
  inflating: /trainA/trainA_pt_1

In [16]:
Fp= glob.glob('/trainB')

In [ ]:
!bash ./datasets/download_cyclegan_dataset.sh horse2zebra

# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_cyclegan_model.sh [apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [ ]:
!bash ./scripts/download_cyclegan_model.sh horse2zebra

# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [17]:
!mv /trainA/trainA_pt_16_128/ /CycleGAN-VAE-NST/datasets/trainA

mv: cannot move '/trainA/trainA_pt_16_128/' to '/CycleGAN-VAE-NST/datasets/trainA': No such file or directory


In [19]:
!python train.py --dataroot ./ --name portraits2op --netG VAEGAN --dataset_mode tensors --crop_size 128 --batch_size 4 --verbose --n_epochs 50 --n_epochs_decay 40 --display_id -1

Traceback (most recent call last):
  File "/content/CycleGAN-VAE-NST/train.py", line 28, in <module>
    opt = TrainOptions().parse()   # get training options
          ^^^^^^^^^^^^^^^^^^^^^^
  File "/content/CycleGAN-VAE-NST/options/base_options.py", line 118, in parse
    opt = self.gather_options()
          ^^^^^^^^^^^^^^^^^^^^^
  File "/content/CycleGAN-VAE-NST/options/base_options.py", line 84, in gather_options
    dataset_option_setter = data.get_option_setter(dataset_name)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/CycleGAN-VAE-NST/data/__init__.py", line 43, in get_option_setter
    dataset_class = find_dataset_using_name(dataset_name)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/CycleGAN-VAE-NST/data/__init__.py", line 36, in find_dataset_using_name
    raise NotImplementedError("In %s.py, there should be a subclass of BaseDataset with class name that matches %s in lowercase." % (dataset_filename, target_da

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [ ]:
!python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_fake.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_real.png')
plt.imshow(img)